Consigna:
def userdata(User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.
Ejemplo de retorno: {"Usuario X" : us213ndjss09sdf, "Dinero gastado": 200 USD, "% de recomendación": 20%, "cantidad de items": 5}

In [9]:
import json
import pandas as pd
import gzip
import ast
from datetime import datetime
import numpy as np

In [ ]:
''' Lectura de Archivos '''
# Se abren los 3 dataframes por que el data frame de la función necesita por lo menos un dato de cada uno

df_steam_games = r"C:\Users\Usuario\Henry\PI1_ML\DataSet-EDAs\df_steam_games.parquet"
df_steam_games = pd.read_parquet(df_steam_games )
df_steam_games

In [ ]:
df_user_reviews = r"C:\Users\Usuario\Henry\PI1_ML\DataSet-EDAs\df_user_reviews.parquet"
df_user_reviews = pd.read_parquet(df_user_reviews)
df_user_reviews

In [ ]:
df_users_items = r"C:\Users\Usuario\Henry\PI1_ML\DataSet-EDAs\df_users_items.parquet"
df_users_items = pd.read_parquet(df_users_items)
df_users_items 

In [ ]:
# Inicio el dataframe de la función por user_items

df_userdata = df_users_items
df_userdata

In [ ]:
df_userdata.info()


In [15]:
del df_users_items

In [8]:
# Cambio el tipo de playtime forever de float 63 a float 32 por espacio

#df_userdata['playtime_forever'] = df_userdata['playtime_forever'].astype('float32')

In [ ]:
df_userdata.info()

In [17]:
# De este df solo necesito user_id, item_id, 'items_count', item_name y play time_forever, así que

columnas_borrar = [ 'user_url', 'playtime_2weeks'] 

df_userdata = df_userdata.drop(columnas_borrar,axis=1)

In [ ]:
df_userdata.head()

In [ ]:
# Reviso Stam_Games pues lo relacionare con el precio 
df_steam_games.head()

In [ ]:
# De steam_games solo necesito item_id y price asi que borro las demás columnas
# Item_id = app_name

columnas_borrar2 = ['publisher', 'genres', 'url', 'release_date', 'reviews_url', 'specs', 'id', "early_access", 'developer', 'release_year'] 

df_steam_games = df_steam_games .drop(columnas_borrar2, axis=1)

df_steam_games

In [ ]:
df_steam_games.head()

In [22]:
# Cambio nombre de columna app_name por "item_name" para que pueda hacer el match

df_steam_games = df_steam_games.rename(columns={'app_name': 'item_name'})


In [ ]:
# reviso la columna price de steam-games

df_steam_games.info()

In [21]:
# Modifico de una vez la columna price para que se pueda sumar luego

#df_steam_games['price'] = pd.to_numeric(df_steam_games['price'], errors='coerce')
#df_steam_games = df_steam_games.dropna(subset=['price'])

In [ ]:
df_steam_games.info()

In [23]:
# Como no me dejó convertir Convierto de float 64 a float 32 por temas de espacio, me toca  crear arreglo y convertirlo luego
#así no me djejo:  df_steam_games['price'] .astype(np.float32)

# rreglo = df_steam_games['price'] .to_numpy() 

In [24]:
# convierto el arreglo en float 32

#df_float32 = arreglo.astype('float32')

In [25]:
#df_float32

array([4.99, 4.99, 4.99, ..., 4.99, 4.99, 4.99], dtype=float32)

In [26]:
# Como ya tengo arreglo en float 32, voy a cambar la columna price por esta en steam_games

#df_steam_games['price'] = df_float32


In [ ]:
df_steam_games.info()

In [26]:
# Relaciono steam_games con userdata a travez de item_name para aumentar la coloumna  precio al dataframe

df_userdata = pd.merge(df_userdata, df_steam_games[['item_name', 'price']], on='item_name', how='left')  


In [ ]:
df_userdata.info()

In [28]:
df_userdata.shape

(32746611, 6)

In [29]:
del df_steam_games

In [ ]:
df_userdata.info()

In [31]:
df_userdata = df_userdata.dropna()

In [ ]:
df_userdata.info()

In [33]:
# retiro duplicados

df_userdata = df_userdata.drop_duplicates()


In [ ]:
df_userdata.info()

In [ ]:
df_user_reviews.head()

In [36]:
# solo necesito item_id y recommend  dereviewspor lo tanto borraré las otras columnas

columnas_borrar3 = ['user_id', 'user_url', 'posted', 'sentiment_analysis' ]

In [37]:
df_user_reviews = df_user_reviews.drop(columnas_borrar3, axis=1)

In [ ]:
df_user_reviews.head()

In [ ]:
df_user_reviews.info()

In [40]:
df_user_reviews.shape

(59305, 2)

In [41]:
df_user_reviews = df_user_reviews.dropna()

In [42]:
df_user_reviews = df_user_reviews.drop_duplicates()

In [ ]:
df_user_reviews.head()

In [ ]:
df_userdata.head()

In [45]:
# Solo queda faltando la columna recommend de user reviews y hare la relación a travez de item_id}

df_userdata = pd.merge(df_userdata, df_user_reviews[['item_id', 'recommend']], on='item_id', how='left')  


In [48]:
df_userdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3856556 entries, 0 to 3856555
Data columns (total 7 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   item_id           object 
 3   item_name         object 
 4   playtime_forever  float64
 5   price             object 
 6   recommend         object 
dtypes: float64(1), int64(1), object(5)
memory usage: 206.0+ MB


In [49]:
# Paso el tipo de price de object a float
df_userdata['price'] = df_userdata['price'].astype(float)



In [ ]:
df_userdata.info()

In [51]:
# Ahora para la función necesito columna costo que creare multiplicando playtime forever y price y luego hago la columna del porcentaje

df_userdata['cantidad total gastado'] = df_userdata['playtime_forever'] * df_userdata['price']

In [ ]:
df_userdata.head()

In [53]:
# Tambien necesito columna del porcentaje (Hago columna del porcentaje de registros del mismo item con true) para eso primero cantidad de registros por item_id y tiene une excepcion para que no divida por cero

porcentaje_true = (df_userdata['recommend'].mean() * 100).round(2)


In [54]:
df_userdata['percentage_true'] = porcentaje_true

In [ ]:
df_userdata.info()
# Claramente la columna funciona

In [56]:
# Borro las columnas que no me sirven o que ya utilice

columnas_borrar4 = ["items_count", "item_name", "playtime_forever", "price", "recommend"]

In [57]:
df_userdata = df_userdata.drop(columnas_borrar4, axis=1)

In [58]:
df_userdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3856556 entries, 0 to 3856555
Data columns (total 4 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   user_id                 object 
 1   item_id                 object 
 2   cantidad total gastado  float64
 3   percentage_true         float64
dtypes: float64(2), object(2)
memory usage: 117.7+ MB


In [59]:
#ahora tengo que cambiar el tipo del porcentaje de float 64 a str

df_userdata['percentage_true'] = df_userdata['percentage_true'].astype(str)


In [ ]:
df_userdata.info()

In [ ]:
df_userdata.to_parquet('df_userdata.parquet')
df_userdata